# Cheat Sheet: Conceptos Introductorios de Modelado Supervisado

Cuaderno ampliado con explicaciones paso a paso sobre métricas básicas, curvas ROC/PR y la intuición del trade-off sesgo-varianza utilizando ejemplos reproducibles.

## Índice Guiado
- [0. Contexto](#0-contexto)
- [1. Preparación del entorno](#1-preparación-del-entorno)
  - [1.1. Librerías y configuración](#11-librerías-y-configuración)
- [2. Error cuadrático medio y correlación](#2-error-cuadrático-medio-y-correlación)
  - [2.1. Simulación de datos lineales](#21-simulación-de-datos-lineales)
  - [2.2. Métricas básicas](#22-métricas-básicas)
  - [2.3. Visualización y lectura conjunta](#23-visualización-y-lectura-conjunta)
- [3. Curvas ROC y Precision-Recall](#3-curvas-roc-y-precision-recall)
  - [3.1. Construcción de etiquetas y scores](#31-construcción-de-etiquetas-y-scores)
  - [3.2. Gráfica ROC](#32-gráfica-roc)
  - [3.3. Gráfica Precision-Recall](#33-gráfica-precision-recall)
- [4. Limitaciones de la accuracy y MCC](#4-limitaciones-de-la-accuracy-y-mcc)
- [5. Trade-off Sesgo-Varianza](#5-trade-off-sesgo-varianza)
  - [5.1. Datos y función verdadera](#51-datos-y-función-verdadera)
  - [5.2. Predictores simples repetidos](#52-predictores-simples-repetidos)
  - [5.3. Modelos polinómicos de grado 3](#53-modelos-polinómicos-de-grado-3)
  - [5.4. Modelos polinómicos de grado 10](#54-modelos-polinómicos-de-grado-10)
- [6. Conclusiones y sugerencias](#6-conclusiones-y-sugerencias)

## 0. Contexto
El ejercicio original muestra métricas fundamentales para modelos de regresión y clasificación, así como la intuición detrás del trade-off sesgo-varianza. Aquí se documenta cada paso para comprender qué calcula cada bloque de código y cómo interpretar las salidas.

## 1. Preparación del entorno

### 1.1. Librerías y configuración
Importamos dependencias para manipulación numérica, evaluación de modelos y visualización. También fijamos semillas para reproducibilidad.

In [ ]:
# Dependencias de numpy y scikit-learn para métricas y curvas
import numpy as np
import random

import matplotlib.pyplot as plt
import pylab as pl
from sklearn import metrics
from sklearn.metrics import matthews_corrcoef
from random import sample

SEED = 123
np.random.seed(SEED)
random.seed(SEED)

print(f"Entorno cargado. Semillas fijadas en {SEED}.")

## 2. Error cuadrático medio y correlación
Introducimos un ejemplo sencillo de regresión lineal con ruido para ilustrar el cálculo de MSE y correlación de Pearson entre valores verdaderos y predichos.

### 2.1. Simulación de datos lineales
Creamos puntos equiespaciados, generamos una relación lineal y añadimos ruido gaussiano para simular predicciones.

In [ ]:
n_points = 51
x = np.linspace(0, 5, n_points)
y_true = 3 * x

# Añadimos ruido gaussiano para emular predicciones imperfectas
pred_with_noise = y_true + np.random.normal(loc=0, scale=1, size=n_points)

print(f"Primeros valores reales: {y_true[:5]}")
print(f"Primeros valores predichos: {pred_with_noise[:5]}")

### 2.2. Métricas básicas
Calculamos el error cuadrático medio (MSE) y la correlación de Pearson para cuantificar precisión y alineación de tendencias.

In [ ]:
mse_value = np.mean((y_true - pred_with_noise) ** 2)
correlation_matrix = np.corrcoef(y_true, pred_with_noise)
pearson_corr = correlation_matrix[0, 1]

print(f"MSE: {mse_value:.4f}")
print(f"Correlación de Pearson: {pearson_corr:.4f}")

### 2.3. Visualización y lectura conjunta
Comparamos `y_true` vs. `pred_with_noise`, anotando MSE y correlación dentro de la gráfica para interpretar ambos valores de manera simultánea.

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_true, pred_with_noise, alpha=0.7, label='Predicción vs. realidad')
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', label='Línea ideal')
plt.xlabel('Valor real (y_true)')
plt.ylabel('Predicción con ruido')
plt.title(f'y vs pred\nMSE={mse_value:.2f}, cor={pearson_corr:.2f}')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

## 3. Curvas ROC y Precision-Recall
Cuando evaluamos clasificadores binarios, las curvas ROC y Precision-Recall permiten estudiar el desempeño a distintos umbrales.

### 3.1. Construcción de etiquetas y scores
Generamos una serie de etiquetas desequilibradas y puntuaciones simuladas para emular la salida de un clasificador probabilístico.

In [ ]:
n_samples = 100
positives_target = 5
candidate_pool = 15

labels = np.zeros(n_samples, dtype=int)
pos_indices = sample(range(1, candidate_pool + 1), positives_target)
labels[pos_indices] = 1

raw_scores = np.sort(np.random.normal(loc=0, scale=1, size=n_samples))[::-1]

print(f"Proporción positiva: {labels.mean():.2f}")

### 3.2. Gráfica ROC
Calculamos la tasa de verdaderos positivos vs. falsos positivos a distintos umbrales, junto con el área bajo la curva (AUC).

In [ ]:
fpr, tpr, roc_thresholds = metrics.roc_curve(labels, raw_scores)
roc_auc_value = metrics.auc(fpr, tpr)

pl.clf()
pl.plot(fpr, tpr, label='Curva ROC')
pl.plot([0, 1], [0, 1], 'k--', label='Azar')
pl.xlabel('Tasa de falsos positivos (FPR)')
pl.ylabel('Tasa de verdaderos positivos (TPR)')
pl.title(f'Curva ROC : AUC={roc_auc_value:.2f}')
pl.ylim([0.0, 1.05])
pl.xlim([0.0, 1.0])
pl.legend(loc='lower right')
pl.show()

### 3.3. Gráfica Precision-Recall
Útil en escenarios con clases desbalanceadas para observar el compromiso entre precisión y recall.

In [ ]:
precision, recall, pr_thresholds = metrics.precision_recall_curve(labels, raw_scores)
pr_auc_value = metrics.auc(recall, precision)

pl.clf()
pl.plot(recall, precision, label='Precision-Recall')
pl.xlabel('Recall')
pl.ylabel('Precision')
pl.title(f'Curva Precision-Recall : AUC={pr_auc_value:.2f}')
pl.ylim([0.0, 1.05])
pl.xlim([0.0, 1.0])
pl.legend(loc='lower left')
pl.show()

## 4. Limitaciones de la accuracy y MCC
La exactitud puede ser engañosa en datasets desbalanceados. Usamos el coeficiente de correlación de Matthews (MCC) para capturar equilibrio entre clases.

In [ ]:
n_total = 1000
n_positive = 100
predicted_positives = 50

actual_labels = np.zeros(n_total, dtype=int)
predicted_labels = np.zeros(n_total, dtype=int)

actual_labels[:n_positive] = 1
candidate_indices = list(range(n_positive + predicted_positives))
pred_indices = sample(candidate_indices, predicted_positives)
predicted_labels[pred_indices] = 1

accuracy_value = np.mean(actual_labels == predicted_labels)
mcc_value = matthews_corrcoef(actual_labels, predicted_labels)

print(f"Accuracy: {accuracy_value:.4f}")
print(f"Matthews Corrcoef: {mcc_value:.4f}")

**Interpretación:** Aunque la accuracy puede verse alta debido a la abundancia de ceros bien clasificados, el MCC penaliza tanto falsos positivos como falsos negativos, reflejando mejor la falta de equilibrio.

## 5. Trade-off Sesgo-Varianza
Visualizamos cómo estimadores simples (alta varianza) y modelos más complejos (posible alto sesgo) se comportan al ajustar datos ruidosos.

### 5.1. Datos y función verdadera
Construimos un conjunto con estructura polinómica cúbica y ruido para comparar modelos con distintas complejidades.

In [ ]:
from scipy.interpolate import UnivariateSpline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

x_raw = np.arange(0, 5.1, 0.1)
x = np.repeat(x_raw, 3)
y_observed = 0 + 2 * x - 7.5 * x ** 2 + 1.5 * x ** 3 + np.random.normal(0, 4, len(x))
true_y = 0 + 2 * x - 7.5 * x ** 2 + 1.5 * x ** 3

plt.figure(figsize=(8, 5))
plt.scatter(x, y_observed, s=20, alpha=0.7, label='Datos observados')
plt.plot(x, true_y, 'r-', linewidth=2, label='Función verdadera')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Datos originales vs. función verdadera')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.2. Predictores simples repetidos
Estimamos promedio sobre muestras aleatorias para mostrar alta varianza entre modelos extremadamente simples.

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(x, y_observed, s=20, alpha=0.4, label='Datos observados')
plt.plot(x, true_y, 'r-', linewidth=2, label='Función verdadera')

sample_size = 100
mean_prediction = np.mean(y_observed[sample(range(len(x)), sample_size)])
plt.axhline(y=mean_prediction, color='#0000AA', linewidth=2, label='Predicción media (muestra 1)')

for _ in range(4):
    mean_prediction = np.mean(y_observed[sample(range(len(x)), sample_size)])
    plt.axhline(y=mean_prediction, color='#0000AA', linewidth=1.5, alpha=0.3)

plt.xlabel('x')
plt.ylabel('y')
plt.title('Predictores extremadamente simples (alto sesgo)')
plt.legend(loc='upper left')
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.3. Modelos polinómicos de grado 3
Entrenamos varias regresiones polinómicas cúbicas sobre subconjuntos aleatorios para observar estabilidad razonable y ligera variación.

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(x, y_observed, color='steelblue', s=30, alpha=0.4, label='Datos observados')
plt.plot(x, true_y, 'r-', linewidth=2, label='Función verdadera')

poly_deg3 = PolynomialFeatures(degree=3)
x_curve = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
x_curve_poly = poly_deg3.fit_transform(x_curve)

for _ in range(5):
    idx_train = sample(range(len(x)), 100)
    x_train = x[idx_train].reshape(-1, 1)
    y_train = y_observed[idx_train]
    x_poly = poly_deg3.fit_transform(x_train)
    model = LinearRegression().fit(x_poly, y_train)
    y_pred_curve = model.predict(x_curve_poly)
    plt.plot(x_curve, y_pred_curve, color='#AA000060', linewidth=2)

plt.xlabel('x')
plt.ylabel('y')
plt.title('Regresión polinómica grado 3 (varias muestras)')
plt.legend(loc='upper left')
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.4. Modelos polinómicos de grado 10
Al aumentar la complejidad, observamos posibles sobreajustes y oscilaciones extremas que ilustran alta varianza.

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(x, y_observed, color='steelblue', s=30, alpha=0.4, label='Datos observados')
plt.plot(x, true_y, 'r-', linewidth=2, label='Función verdadera')

poly_deg10 = PolynomialFeatures(degree=10)
x_curve_poly10 = poly_deg10.fit_transform(x_curve)

for _ in range(5):
    idx_train = sample(range(len(x)), 100)
    x_train = x[idx_train].reshape(-1, 1)
    y_train = y_observed[idx_train]
    x_poly = poly_deg10.fit_transform(x_train)
    model = LinearRegression().fit(x_poly, y_train)
    y_pred_curve = model.predict(x_curve_poly10)
    plt.plot(x_curve, y_pred_curve, color='#00800060', linewidth=2)

plt.xlabel('x')
plt.ylabel('y')
plt.title('Regresión polinómica grado 10 (varias muestras)')
plt.legend(loc='upper left')
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Conclusiones y sugerencias
- El MSE captura magnitud de error, mientras que la correlación refleja alineación de tendencias.
- Las curvas ROC y Precision-Recall complementan la evaluación, especialmente en escenarios desbalanceados.
- La accuracy puede ocultar errores críticos; MCC ofrece una visión equilibrada.
- El trade-off sesgo-varianza se manifiesta en la estabilidad de las curvas polinómicas: complejidad baja puede sub-ajustar, complejidad alta sobre-ajusta.
- Siguientes pasos: evaluar métricas adicionales (F1, balanced accuracy), experimentar con regularización o técnicas de validación cruzada.